In [1]:
!nvidia-smi

Fri Jul 25 05:29:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/ziye2chen/LLMs-for-Mathematical-Analysis.git

Cloning into 'LLMs-for-Mathematical-Analysis'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 84 (delta 34), reused 53 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 7.71 MiB | 6.16 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Error downloading object: MathAnalysis_Llama_Classifier/adapter_model.safetensors (601a046): Smudge error: Error downloading MathAnalysis_Llama_Classifier/adapter_model.safetensors (601a0467f98c5d7a7887a6922be2379af510dcd3e5d87f274500966cff2e7b9b): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/LLMs-for-Mathematical-Analysis/.git/lfs/logs/20250725T053002.919496658.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: MathAnalysis_Llama_Classifier/adapter_model.safetensors

In [3]:
!pip install --upgrade pip
!pip install unsloth transformers trl datasets pandas matplotlib rich scipy sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 135.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 132.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 74.0 MB/s 

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [6]:
import torch
print("Pytorch version：")
print(torch.__version__)
print("CUDA Version: ")
print(torch.version.cuda)
print("cuDNN version is :")
print(torch.backends.cudnn.version())

Pytorch version：
2.7.1+cu126
CUDA Version: 
12.6
cuDNN version is :
90501


In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.7.8: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/Qwen2.5-Math-7B-bnb-4bit can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 2.5, it can be magically be extended to 10240!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Classifier

In [12]:
alpaca_prompt = """As a mathematical assistant, You need to analyze the problem to find out what type of problem it belongs to in Real Analysis.

### Problem:
{}

### Problem_Type:
{}"""


In [14]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    Problem_Type = examples["problem_type"]
    Problem = examples["problem"]

    texts = []
    for problem, problem_type in zip(Problem, Problem_Type):
        text = alpaca_prompt.format(problem, problem_type) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}


from datasets import load_dataset
dataset = load_dataset('csv',data_files = '/content/train.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/901 [00:00<?, ? examples/s]

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 300, # 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"]:   0%|          | 0/901 [00:00<?, ? examples/s]

In [16]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.822 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 901 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Step,Training Loss
1,2.534100
2,2.761000
3,2.454000
4,2.053900
5,2.413500
6,2.293300
7,2.030700
8,2.004700
9,1.823500
10,1.190800


Unsloth: Will smartly offload gradients to save VRAM!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [20]:
model.save_pretrained("MathAnalysis_Qwen_Classifier") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_Classifier")
# model.save_pretrained("/content/drive/MyDrive/MathAnalysis_Qwen_Classifier") # Local saving
# tokenizer.save_pretrained("/content/drive/MyDrive/MathAnalysis_Qwen_Classifier")

('/content/drive/MyDrive/MathAnalysis_Qwen_Classifier/tokenizer_config.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_Classifier/special_tokens_map.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_Classifier/vocab.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_Classifier/merges.txt',
 '/content/drive/MyDrive/MathAnalysis_Qwen_Classifier/added_tokens.json',
 '/content/drive/MyDrive/MathAnalysis_Qwen_Classifier/tokenizer.json')

In [21]:
test_prompt = """Là trợ lý toán học, bạn cần phân tích bài toán để tìm ra loại bài toán đó trong Phân tích Thực tế. Cung cấp Problem_Type và Kiến thức có thể được sử dụng để giải bài toán này.

### Problem:
{}
"""

In [23]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_Classifier", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    test_prompt.format(
        "Cho các số thực dương a, b, c thỏa abc = 2. Chứng minh rằng: a^2 + b^2 + c^2 - 3(a + b - c) \\\\ge \\\\frac{9}{4}.", # Problem
    )
], return_tensors = "pt").to("cuda")


==((====))==  Unsloth 2025.7.8: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [24]:
# Tự tính max_new_tokens an toàn
input_len = inputs.input_ids.shape[-1]
max_total_length = max_seq_length  # ví dụ: 10240
safe_max_new_tokens = max_total_length - input_len

# In thử kiểm tra
print(f"Input tokens: {input_len}, Max new tokens: {safe_max_new_tokens}")

# Dùng TextStreamer để in ra từng dòng sinh token
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Sinh kết quả
_ = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    streamer=text_streamer,
    max_new_tokens=safe_max_new_tokens,
    pad_token_id=tokenizer.eos_token_id
)

Input tokens: 101, Max new tokens: 10139
### Problem_Type:
Bất đẳng thức<|endoftext|>


In [28]:
import pandas as pd
from unsloth import FastLanguageModel
from transformers import TextStreamer

# ==== 1️⃣ Đọc file CSV chứa 100 câu ====
df = pd.read_csv("test.csv")  # Đảm bảo file có cột: Problem
outputs = []

print(f"Tổng số câu: {len(df)}")

# ==== 2️⃣ Prompt template ====
test_prompt = """Là trợ lý toán học, bạn cần phân tích bài toán để tìm ra loại bài toán đó trong Phân tích Thực tế. Cung cấp Problem_Type.

### Problem:
{}
"""

# ==== 3️⃣ Load model ====
model_name = "MathAnalysis_Qwen_Classifier"
max_seq_length = 4096
dtype = torch.float16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_Classifier", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

# ==== 4️⃣ Streamer để hiển thị từng token ====
streamer = TextStreamer(tokenizer, skip_prompt=True)

# ==== 5️⃣ Lặp qua từng bài ====
for idx, row in df.iterrows():
    problem = row["Problem"]
    prompt = test_prompt.format(problem)

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    input_len = inputs.input_ids.shape[-1]
    safe_max_new_tokens = max_seq_length - input_len

    streamer = TextStreamer(tokenizer, skip_prompt=True)
    generated = model.generate(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
        max_new_tokens = safe_max_new_tokens,
        pad_token_id = tokenizer.eos_token_id,
    )

    # Lấy text ra (dạng string)
    output_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    outputs.append({
        "Problem": problem,
        "Predicted_ProblemType": output_text
    })

# Sau vòng lặp — lưu file CSV
df_out = pd.DataFrame(outputs)
df_out.to_csv("test_results.csv", index=False)
print("✅ Đã lưu kết quả vào test_results.csv")


Tổng số câu: 100
==((====))==  Unsloth 2025.7.8: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

# Problem Solver

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "auto",
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False,  # ← TẮT ở đây
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
alpaca_prompt = """As a mathematical assistant, solve the following problem. Provide a detailed, step-by-step solution using rigorous mathematical reasoning. If the problem requires the use of the $\epsilon$-$\delta$ method (e.g., when proving limits or continuity), ensure that you apply it appropriately. Use precise mathematical language and notation throughout your solution.

### Problem_Type:
{}

### Problem:
{}

### Solution:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    Problem_Type    = examples["ProblemType"]
    Problem       = examples["Problem"]
    Solution  = examples["Solution"]
    texts = []
    for problem_type, problem, solution in zip(Problem_Type, Problem, Solution):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(problem_type, problem, solution) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv',data_files = '/content/LLMs-for-Mathematical-Analysis/pretraining_data.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
len(dataset)

49

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 300, # 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"]:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.824 GB of memory reserved.


In [ ]:

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 49 | Num Epochs = 12 | Total steps = 300
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.836000
2,0.880100
3,1.096700
4,0.850100
5,0.901600
6,0.937500
7,0.825100
8,0.841000
9,0.653100
10,0.745700


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
model.save_pretrained("MathAnalysis_Qwen_ProblemSolver") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_ProblemSolver")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('MathAnalysis_Qwen_ProblemSolver/tokenizer_config.json',
 'MathAnalysis_Qwen_ProblemSolver/special_tokens_map.json',
 'MathAnalysis_Qwen_ProblemSolver/chat_template.jinja',
 'MathAnalysis_Qwen_ProblemSolver/tokenizer.model',
 'MathAnalysis_Qwen_ProblemSolver/added_tokens.json',
 'MathAnalysis_Qwen_ProblemSolver/tokenizer.json')

In [ ]:
# PROMPT: dùng để sinh lời giải bài toán
problem_prompt = """As a mathematical assistant, you need to solve the following Real Analysis problem. Provide a detailed proof.

### Problem:
{}

### Problem_Type:
{}
"""

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "MathAnalysis_Qwen_ProblemSolver",  # Đổi sang mô hình ProblemSolver
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)  # Tăng tốc inference

# Nhập bài toán cụ thể
inputs = tokenizer(
[
    problem_prompt.format(
        "Suppose that $a_n$ and $b_n$ converge to $\alpha$ and $\beta$ as $n \to \infty$ respectively. Show that the sequence $$ \frac{a_0 b_n + a_1 b_{n-1} + \cdots + a_n b_0}{n} $$ converges to $\alpha \beta$ as $n \to \infty$.",  # Câu hỏi LaTeX hoặc plain text
        "Sequences and Limits", # Problem_Type
    )
], return_tensors = "pt").to("cuda")

In [ ]:
# Tính độ dài đầu vào an toàn
input_len = inputs.input_ids.shape[-1]
max_total_length = max_seq_length  # ví dụ: 10240
safe_max_new_tokens = max_total_length - input_len

print(f"Input tokens: {input_len}, Max new tokens: {safe_max_new_tokens}")

# Stream kết quả trả ra từng dòng
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Sinh lời giải
_ = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    streamer=text_streamer,
    max_new_tokens=safe_max_new_tokens,
    pad_token_id=tokenizer.eos_token_id
)